# **AUTOMATIZACIÓN CON SELENIUM PARA LA ACTUALIZACIÓN DE FIRMWARE DE SWITCHS ARUBA 1930 EN LOS ESTABLECIMIENTOS DE MULTINACIONAL DE COMIDA RÁPIDA.**

In [ ]:
#Instalar las dependencias necesarias
!pip install selenium
!pip install webdriver_manager
!pip install selenium cryptography

In [ ]:
#Variables de Entorno
%env CLAVE_GMAIL= nedu gjmr xxos oxar
%env CLAVE_ADMIN= Slapeoar!9907
%env CORREO_GMAIL= kfcpablo@gmail.com
%env USER_SWITCH= admin

In [ ]:
#Script para encriptar la clave de switch - Seguridad
from cryptography.fernet import Fernet
import os

# Generar y guardar una clave
key = Fernet.generate_key()
with open(r'D:\python\switchsllave.key', 'wb') as key_file:
    key_file.write(key)

# Encriptar un mensaje
fernet = Fernet(key)
clave_encriptada = fernet.encrypt(b'Slapeoar!9907')


# Guardar el mensaje encriptado en un archivo
with open(r'D:\python\clave_encriptada.txt', 'wb') as enc_file:
    enc_file.write(clave_encriptada)
    
#Clave encriptada exitosamente
print("Mensaje encriptado correctamente")
print("Clave encriptada:", clave_encriptada)


In [ ]:
#Script para desencriptar la clave de switch
from selenium import webdriver
from cryptography.fernet import Fernet,InvalidToken

# Especificar las rutas de los archivos
key_path = r'D:\python\switchsllave.key'
clave_encriptada_path = r'D:\python\clave_encriptada.txt'
try:
    # Leer la clave de encriptación
    with open(key_path, 'rb') as key_file:
        key = key_file.read()

    # Leer el mensaje encriptado
    with open(clave_encriptada_path, 'rb') as enc_file:
        clave_encriptada = enc_file.read()

    # Desencriptar el mensaje
    fernet = Fernet(key)
    clave_desencriptada = fernet.decrypt(clave_encriptada).decode()

    # Mensaje de confirmación si se desencripta correctamente
    print("Clave encriptada:", clave_encriptada)
    print("Mensaje desencriptado correctamente:", '######')
    #print("Su clave desencriptada es:" , clave_desencriptada )

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada.")
except InvalidToken:
    print(f"Error: No se pudo desencriptar el mensaje. La clave de encriptación podría ser incorrecta o el mensaje está corrupto.")
except Exception as e:
    print(f"Error desconocido: {str(e)}")

In [ ]:
#Script para cargar el Firmware IOS en Switch Aruba 1930
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
import pandas as pd
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

#Función para esperar hasta que el texto 'Succeeded' aparezca en pantalla
def esperar_texto_succeeded(element, timeout=700):
    WebDriverWait(browser, timeout).until(
        EC.text_to_be_present_in_element((By.ID, element), 'Succeeded')
    )

#Arreglo de lista de switchs y status
data = {'Switch': [], 'Status': []}

# Configuración del navegador
CHROMEDRIVER_PATH = "D:\\python\\chromedriver126.0.6478.exe"
path = Service(CHROMEDRIVER_PATH)
browser = webdriver.Chrome(service=path)

#Credenciales y rutas
username = %env USER_SWITCH
iospath = "InstantOn_1930_2.6.0.74.swi"
path = "D:\\python\\switchsaruba2.txt"

with open(path, "r") as file:
    switches = [line.strip() for line in file]

try:
    for ip in switches:
        print(ip)
        try:
            # Abre cada URL por IP
            browser.get('http://' + ip)
            time.sleep(4)
            # Ingresar a la administración
            browser.find_element(By.NAME, "inputUsername").send_keys(username)
            #print (clave_desencriptada)
            browser.find_element(By.NAME, "inputPassword").send_keys(clave_desencriptada)
            browser.find_element(By.CLASS_NAME, "btn").click()
            
            # Obtener la versión de IOS Actual.
            time.sleep(8)
            ver_sw = browser.find_element(By.ID, "lblDeviceVersion")
            text_ver = ver_sw.text
            print(text_ver)
            
            # Si la versión es estable entonces salta al siguiente.
            if '2.6.0' in text_ver:
                print("Ya actualizado")
                data['Switch'].append(ip)
                data['Status'].append('YA_ACTUALIZADO')
                continue
            else:
                print("Continúa revisión")       
        
            # Navegación para la carga del IOS.
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "Maintenance"))).click()
            time.sleep(3)
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "Backup and Update Files"))).click()
            
            # Seleccionando opción Update.
            time.sleep(2)
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, "lblrbOperationType_0"))).click()
            browser.find_element(By.ID, "btnNext").click()
            
            # Seleccionando opción Backup Image.
            time.sleep(2)
            Select(browser.find_element(By.ID, "slctFileType")).select_by_value("1")
            browser.find_element(By.ID, "btnNext").click()
              
            # Seleccionando opción TFTP.
            time.sleep(2)
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, "lblrbTransferProtocol_1"))).click()
            browser.find_element(By.ID, "btnNext").click()
            
            # Seleccionar ruta del IOS por TFTP.
            time.sleep(2)
            browser.find_element(By.ID, "txtServerAddressTFTP").send_keys("192.168.100.123")
            browser.find_element(By.ID, "txtFileNameTFTP").send_keys(iospath)
            
            # Carga de URL con la ruta del IOS.
            time.sleep(2)
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, "buttonTransferProtocolTFTP"))).click()
                     
            # Esperar hasta que el texto 'Succeeded' aparezca
            esperar_texto_succeeded("lblStatusModal")
            status = browser.find_element(By.ID, "lblStatusModal").text
            print(status)
            
            if 'Succeeded' in status:
                # Si el estado es 'Succeeded' en pantalla, hace clic en el botón de Cancelar reinicio
                WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, "modalStatusButtonOk"))).click()
                data['Switch'].append(ip)
                data['Status'].append('OK')
            else:
                data['Switch'].append(ip)
                data['Status'].append('CONFIGURACION_FALLIDA')
            
        except Exception as e:
            print(e)
            data['Switch'].append(ip)
            data['Status'].append('CONECCION_FALLIDA')
     
finally:
    file.close()    
    browser.quit()  # Las instancias de WebDriver se cierren   

#Visualización Reporte de Status en Data Frame
report = pd.DataFrame(data)
print (report)

# Envio de correo Reporte de Carga de Firmware 
# Configuración del servidor SMTP y credenciales
smtp_server = 'smtp.gmail.com'
smtp_port = 587
usuario_correo = %env CORREO_GMAIL
clave_correo = %env CLAVE_GMAIL

# Configuración del correo
de_direccion = %env CORREO_GMAIL
a_direccion = 'pablo.caceres@kfc.com.ec'

# Obtener la fecha y hora actual
now = datetime.now()
fecha_formato = now.strftime("%Y-%m-%d %H:%M:%S")

# Asunto del correo con fecha y hora
subject = f'Automatización Carga de Firmware Finalizado - {fecha_formato}'
body = f"\nReporte de Automatización de Carga de Firmware Aruba 1930 GRUPOKFC\n\n {report.to_string()}\n\nRealizado por Pablo David Cáceres Castro\n\n"

# Crear el mensaje MIME *Extension multiproposito de correo electronico
msg = MIMEMultipart()
msg['From'] = de_direccion
msg['To'] = a_direccion
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

# Validacion y envio el correo
try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(usuario_correo, clave_correo)
    text = msg.as_string()
    server.sendmail(de_direccion, a_direccion, text)
    server.quit()
    print("Correo enviado exitosamente.")
except Exception as e:
    print(f"Error al enviar el correo: {e}")

#Fin de la automatizacion
print("\n\n######################################")
print("#                                    #")
print("#            FINALIZADO              #")
print("#                                    #")
print("######################################")


In [ ]:
# Filtrar los switches que tienen el estado 'OK'
switches_ok = report[report['Status'] == 'OK']['Switch'].tolist()

# Guardar la lista de switches 'OK' en un archivo
with open('D:\\python\\switches_por_reiniciar.txt', 'w') as f:
    for Switch in switches_ok:
        f.write(Switch + '\n')

print("Lista de switches 'OK' guardada en 'D:\\python\\switches_por_reiniciar.txt'")
#enviar a un correo al administrador. Para Reiniciar

In [ ]:
# Script para reiniciar los switches Aruba 1930
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime
from selenium.common.exceptions import TimeoutException

# Arreglo de lista de switches y status
data = {'Switch': [], 'Status': []}

# Iniciar el navegador
path = Service("D:\\python\\chromedriver126.0.6478.exe")
browser = webdriver.Chrome(service=path)

# Leer la lista de switches desde el archivo
switches_ok = []
with open('D:\\python\\switches_por_reiniciar.txt', 'r') as f:
    switches_ok = [line.strip() for line in f.readlines()]

# Función para esperar texto y refrescar
def actualizar_navegador(browser, ip, data, max_attempts=3, wait_time=30):

    attempts = 0
    found = False
    while attempts < max_attempts:
        try:
            WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Aruba Instant On')]"))
            )
            found = True
            print("Switch Aruba 1930 en Linea.")
            break
        except:
            print("Reiniciando Switch Aruba 1930 - Intentando de nuevo...")
            time.sleep(wait_time)
            browser.refresh()
            attempts += 1

    if not found:
        raise Exception("Switch Aruba inactivo después de varios intentos - Comunicarse con el local.")

    print(f"Switch {ip} Actualizado Exitosamente.")
    data['Switch'].append(ip)
    data['Status'].append('OK')


# Reiniciar switches con estado 'OK'
for ip in switches_ok:
    print(ip)
    
    try:
        # Abre cada URL por IP
        browser.get('http://' + ip)
        time.sleep(4)
     
        # Parte 1: Reinicio inicial - Instalación de IOS
        # Ingresar a la administración
        browser.find_element(By.NAME, "inputUsername").send_keys(username)
        browser.find_element(By.NAME, "inputPassword").send_keys(clave_desencriptada)
        browser.find_element(By.CLASS_NAME, "btn").click()
        
        # Navegar a opción de reinicio
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "Maintenance"))).click()
        time.sleep(3)
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "Reset"))).click()
        
        # Confirmar reinicio
        WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "btnReboot"))).click()
        
        try:
            # Validar que uno de los dos botones de reset esté seleccionable
            reboot_button = WebDriverWait(browser, 15).until(
                EC.element_to_be_clickable((By.ID, "modalRebootButtonApply"))
            )
            reboot_button.click()
        except TimeoutException:
            try:
                confirm_reboot_button = WebDriverWait(browser, 15).until(
                    EC.element_to_be_clickable((By.ID, "modalConfirmButtonSaveReboot"))
                )
                confirm_reboot_button.click()
            except TimeoutException:
                print("Ninguno de los botones de reinicio fue encontrado.")

        # Esperar reinicio del switch y actualización pantalla
        print("Esperando a que el switch termine el reinicio Parte 1")
        time.sleep(400)
        browser.refresh()

        # Verificar si el texto "Aruba Instant On" aparece después del reinicio
        actualizar_navegador(browser, ip, data)

    except Exception as e:
        print(f"Error reiniciando el switch {ip} en Parte 1: {e}")
        data['Switch'].append(ip)
        data['Status'].append('CONECCION_FALLIDA')
        continue
       
    try:
        # Parte 2: Instalar Actualización
        # Ingresar a la administración
        time.sleep(10)
        browser.find_element(By.NAME, "inputUsername").send_keys(username)
        browser.find_element(By.NAME, "inputPassword").send_keys(clave_desencriptada)
        browser.find_element(By.CLASS_NAME, "btn").click()
        
        # Actualizar versión
        time.sleep(10)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, "modalInstallUpdate"))).click()
        
        # Esperar reinicio del switch y actualización pantalla
        print("Esperando a que el switch termine el reinicio Parte 2")
        time.sleep(400)
        browser.refresh()

        # Verificar si el texto "Aruba Instant On" aparece después del reinicio
        # Solo actualizar el estado si no fue actualizado en la Parte 1
        if ip not in data['Switch']:
            actualizar_navegador(browser, ip, data)

    except Exception as e:
        print(f"Error reiniciando el switch {ip} en Parte 2: {e}")
        if ip not in data['Switch']:
            data['Switch'].append(ip)
            data['Status'].append('CONECCION_FALLIDA')

browser.quit()


# Visualización Reporte de Status en Data Frame
report = pd.DataFrame(data)
print(report)

# Envio de correo Reporte Final 
# Configuración del servidor SMTP y credenciales
smtp_server = 'smtp.gmail.com'
smtp_port = 587
usuario_correo = %env CORREO_GMAIL
clave_correo = %env CLAVE_GMAIL

# Configuración del correo
de_direccion = %env CORREO_GMAIL
a_direccion = 'pablo.caceres@kfc.com.ec'

# Obtener la fecha y hora actual
now = datetime.now()
fecha_formato = now.strftime("%Y-%m-%d %H:%M:%S")

# Asunto del correo con fecha y hora
subject = f'Automatización Actualización Firmware Aruba 1930 Finalizado - {fecha_formato}'
body = f"\nReporte de Automatización Firmware Actualizado Aruba 1930 GRUPOKFC\n\n {report.to_string()}\n\nRealizado por Pablo David Cáceres Castro\n\n"

# Crear el mensaje MIME *Extension multiproposito de correo electronico
msg = MIMEMultipart()
msg['From'] = de_direccion
msg['To'] = a_direccion
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

# Validacion y envio el correo
try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(usuario_correo, clave_correo)
    text = msg.as_string()
    server.sendmail(de_direccion, a_direccion, text)
    server.quit()
    print("\nCorreo enviado exitosamente.")
except Exception as e:
    print(f"Error al enviar el correo: {e}")

print("\n\n######################################")
print("#                                    #")
print("#            FINALIZADO              #")
print("#                                    #")
print("######################################")
